# EJERCICIOS

In [21]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten

## EJERCICIO 1

¿Cuántas neuronas hacen falta en la capa de salida para clasificar emails entre correo no deseado y correo deseado? 

¿Qué función de activación deberías usar en la capa de salida? 

Si, en cambio, quieres enfrentarte a MNIST, ¿cuántas neuronas necesitas en la capa de salida y qué función de activación deberías usar? 

¿Y si quieres que la red prediga los precios de la vivienda en California?

In [22]:
# 1 porque la salida es binaria 
# Sigmoide en la capa de salida
# Si es multiclase necesitariamo tantas neuronas como clases y softmax en la capa de salida
# Al ser regresion necesitamos una neurona 

## EJERCICIO 2

Entrena un red neuronal profunda con el conjunto de datos MNIST (puedes cargarlo con tf. keras. datasets.minst.load_data()). Intenta conseguir la mejor pruebas en el conjunto de validación  ajustando los hiperparámetros de modo manual. Prueba a buscar una buena tasa de aprendizaje y utiliza la detención temprana (mínimo hay que conseguir un 97%).

Pasos a seguir: 
+ Entrena varios modelos con distintas tasas de aprendizaje
+ Guarda cada modelo en un fichero
+ Carga cada modelo y evalúalo en el conjunto de cargas y quedate con programación con el mejor (bucle que se quede con el mejor), mostrando la tasa de aprendizaje y la exactitud del mejor modelo.

Si no obtienes más de un 97% vuelve a empezar.

In [23]:
df_mnist=tf.keras.datasets.mnist.load_data()

In [24]:
(x_train, y_train), (x_test, y_test) = df_mnist

# Normalizar los datos 
x_train = x_train / 255.0
x_test = x_test / 255.0


# Dividir el conjunto de entrenamiento en entrenamiento y validación
X_train, X_temp, y_train, y_temp = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Dividir el conjunto temporal en validación y prueba
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [25]:
"""
# Limpiar la sesión actual y liberar recursos
tf.keras.backend.clear_session()

tf.random.set_seed(42)

model = Sequential()
model.add( Input(shape=(28, 28)))
model.add(Flatten())
model.add(Dense(300, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))
"""

'\n# Limpiar la sesión actual y liberar recursos\ntf.keras.backend.clear_session()\n\ntf.random.set_seed(42)\n\nmodel = Sequential()\nmodel.add( Input(shape=(28, 28)))\nmodel.add(Flatten())\nmodel.add(Dense(300, activation="relu"))\nmodel.add(Dense(100, activation="relu"))\nmodel.add(Dense(10, activation="softmax"))\n'

In [26]:
#model.summary()

In [27]:
"""
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])
"""

'\noptimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)\nmodel.compile(loss="sparse_categorical_crossentropy",\n              optimizer=optimizer,\n              metrics=["accuracy"])\n'

In [28]:
"""
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',  # Supervisar la precisión del conjunto de validación
    patience=3,              # Detener si no mejora en 3 épocas consecutivas
    restore_best_weights=True  # Restaurar los mejores pesos alcanzados
)
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))
"""

"\nearly_stopping = tf.keras.callbacks.EarlyStopping(\n    monitor='val_accuracy',  # Supervisar la precisión del conjunto de validación\n    patience=3,              # Detener si no mejora en 3 épocas consecutivas\n    restore_best_weights=True  # Restaurar los mejores pesos alcanzados\n)\nhistory = model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))\n"

In [29]:
#model.evaluate(X_test, y_test)

In [30]:
#model.save("Resultados/4-2-1.keras")

In [ ]:
# Función para crear el modelo
def create_model(learning_rate):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(300, activation="relu"),
        Dense(100, activation="relu"),
        Dense(10, activation="softmax")
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Configurar el callback de detención temprana
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Lista de tasas de aprendizaje para probar
learning_rates = [0.01, 0.001, 0.0001]

# Variables para almacenar el mejor modelo
best_model_path = None
best_accuracy = 0
best_learning_rate = None


# Probar diferentes tasas de aprendizaje
for lr in learning_rates:
    print(f"Entrenando modelo con tasa de aprendizaje: {lr}")
    model = create_model(learning_rate=lr)
    model_path = f"Resultados/model_lr_{lr}.keras"

    history = model.fit(
        X_train, y_train,
        epochs=15,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
    )

    # Guardar el modelo entrenado
    model.save(model_path)

    # Evaluar el modelo en el conjunto de prueba
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Tasa de aprendizaje: {lr}, Precisión en prueba: {test_accuracy * 100:.2f}%")

    # Guardar el mejor modelo
    if test_accuracy > best_accuracy:
        best_model_path = model_path
        best_accuracy = test_accuracy
        best_learning_rate = lr

# Cargar y mostrar los resultados del mejor modelo
best_model = tf.keras.models.load_model(best_model_path)
best_test_loss, best_test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"\nMejor modelo:")
print(f"Tasa de aprendizaje: {best_learning_rate}")
print(f"Precisión en prueba: {best_test_accuracy * 100:.2f}%")

Entrenando modelo con tasa de aprendizaje: 0.01
Epoch 1/15
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8746 - loss: 0.4105 - val_accuracy: 0.9402 - val_loss: 0.2098
Epoch 2/15
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9497 - loss: 0.1863 - val_accuracy: 0.9562 - val_loss: 0.1701
Epoch 3/15
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9580 - loss: 0.1584 - val_accuracy: 0.9472 - val_loss: 0.2262
Epoch 4/15
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9629 - loss: 0.1460 - val_accuracy: 0.9515 - val_loss: 0.2907
Epoch 5/15
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9658 - loss: 0.1412 - val_accuracy: 0.9470 - val_loss: 0.2274
Tasa de aprendizaje: 0.01, Precisión en prueba: 95.23%
Entrenando modelo con tasa de aprendizaje: 0.001
Epoch 1/15
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8873 - loss: 0.3843 - val_accuracy: 0.9657 - val_loss: 0.1129
Epoch 2/15
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.

## EJERCICIO 3

Clasificación de Imágenes de Dígitos Escritos a Mano:

* Objetivo: Entrenar una red neuronal para clasificar imágenes de dígitos escritos a mano utilizando el conjunto de datos SVHN (Street View House Numbers).

* Tareas: 
    * Cargar el conjunto de datos, dividir los datos (train, valid y test), explorar los datos representando alguna imagen con su etiqueta y muestra los valores (únicos) de y (en el entrenamiento, en validación y en pruebas), preparar los datos (d normalizar dividiendo entre 255.0 e igual hay que hacer algo más), construir, compilar y entrenar una red neuronal. 

    * Utilizar detección temprana

    * Hacer una gráfica de como evoluciona accuracy y val_accuracy durante el entrenamiento

    * Si es necesario ajustar el modelo para lograr una mayor precisión.

    * Evalúar el modelo en el conjunto de test y hacer alguna predicción (representa alguna imagen con su etiqueta real y la predicha)

In [14]:
def load_svhn_data(url):
    path = tf.keras.utils.get_file("svhn.mat", url)
    data = loadmat(path)
    return np.transpose(data["X"], (3, 0, 1, 2)), data["y"].flatten()

# Cargar datos
train_url = "http://ufldl.stanford.edu/housenumbers/train_32x32.mat"
test_url = "http://ufldl.stanford.edu/housenumbers/test_32x32.mat"
X_train_full, y_train_full = load_svhn_data(train_url)
X_test, y_test = load_svhn_data(test_url)

182040794/182040794 ━━━━━━━━━━━━━━━━━━━━ 43s 0us/step


## EJERCICIO 4

Predicción del consumo de energía:

* Objetivo: Construir una red neuronal para predecir el consumo de energía de edificios a partir de características como la temperatura, humedad, y tiempo del conjunto de datos Energy Efficiency de UCI Machine Learning Repository

* Tareas: 
    * Cargar el conjunto de datos, dividir los datos (train, valid y test), explorar los datos, preparar los datos (normalizar con una capa de normalización), construir un modelo de regresión, compilar y entrenar una red neuronal. 
    
    * Usa X1-X8 para la X e Y1-Y2 para la y

    * Ajusta el modelo hasta encontrar un error absoluto medio aceptable

    * Haz una gráfica de como evoluciona el error absoluto medio y error absoluto medio del conjunto de validación durante el entrenamiento

    * Evalúa el modelo en el conjunto de test y haz alguna predicción (muestra el valor predicho y el real)

In [16]:
# Descargar el conjunto de datos de eficiencia energética de UCI
url = ("https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx")
data = pd.read_excel(url)